![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)



## 3D Printing

This short notebook demonstrates how to plot a surface in 3D and convert it into an STL file suitable for printing on any standard 3D printer. 

There are four sections to the notebook
- create an STL file for a pyramid
- create an STL file for a cube
- create an STL file for a hyperboloid, which is an example of a surface in 3D
- view the files, and print on a 3D printer

We are inspired by the example here: https://micronote.tech/2020/12/Generating-STL-Models-with-Python/, which shows how to use the basic STL library in Python. The cube example comes from this webpage. 


We start by installint the numpy-stl library, then importing it and a few other necessary libraries. 

In [ ]:
!pip install numpy-stl

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from stl import mesh

## First example. A Pyramid.

To create a file for 3D printing, we start with a solid object and break up its surface into a collection of triangles. 

The simplest example is a symmetric pyramid, with three vertices on the base, and the fourth vertex lying above the base. Each face of the pyramid is a triangle, so there is a natural way to break up this figure into four triangles.
<div align="center">
<img src="images/Pyramid.png" alt="A pyramid with four vertices" width="400"/><br>
A symmetric pyramid, with labelled vertices.
</div>

In this example, we can list the four faces as a list of triangles with three vertices each, like this:
- [0,1,2] - the front triangle, seen in the diagram above)
- [0,2,3] - the back triangle
- [0,3,1] - the side triangle
- [1,3,2] - the bottom triangle

The order of the vertices is important. In each case, the points go counter-clockwise, as seen from the outside of the pyramid. (For the bottom surface, imagine looking from underneath the pyramid.)

We also need to list the four vertices in (x,y,z) coordinates. We put two on the base at (1,0,0) and (-1,0,0). The third point on the base will have x and z coordinates equal to zero, and y has to be set to square root of three, to give an equilateral triangle. This gives the point (0, 1.732, 0). 

The fourth point at the top of the pyramid also has to be a distance of 2 from the other points. A bit of algebra shows the point 
$ (0, 1/\sqrt{3},\sqrt{8/3}) = (0,.577,1.633) $ works. 

We code this into two arrays listing the vertices and the faces, then create a mesh representing the shape as a set of vectors. The last step is to save the data as an STL file. The code is as follows:


In [ ]:
# Define the 4 faces that make up the pyramid, in terms of the numbered vertices
faces = np.array([\
    [0,1,2],
    [0,2,3],
    [0,3,1],
    [1,3,2] 
    ])

# Define the 4 vertices of a pyramid, sides of length 2
vertices = np.array([\
    [ 0, .577, 1.633],  #0
    [-1, 0, 0],    #1
    [ 1, 0, 0],    #2
    [ 0, 1.732, 0], #3
    ])

# Create the mesh
pyramid = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
for index, face in enumerate(faces):
    for j in range(3):
        pyramid.vectors[index][j] = vertices[face[j],:]

# Write the mesh to file "pyramid.stl"
 pyramid.save('pyramid.stl')

## Second example. A cube.

Our next example is a cube, with four vertices on the base, and another four vertices on the layer above. Each face of the cube is a square, which we can break up into two triangles each.
<div align="center">
<img src="images/Cube.png" alt="A cube with eight vertices" width="400"/><br>
A cube, with labelled vertices.
</div>

For the cube, we can list the six faces as pairs of triangles with three vertices each, like this:
-    [0,3,1], [1,3,2], - bottom face
-    [4,5,6], [4,6,7], - top face
-    [0,4,7], [0,7,3], - left front face
-    [0,1,5], [0,5,4], - right front face
-    [2,3,6], [3,7,6], - left back face
-    [5,1,2], [5,2,6]  - right back face

Again, the order of the three vertices in each triangle is important. In each case, the points go counter-clockwise, as seen from the outside of the cube. (For the bottom surface, imagine looking from underneath the cube.)

We also need to list the eight vertices in (x,y,z) coordinates. We set each coordinate to one of -1 or +1, giving a set of eight vertices in all. 

We code this into two arrays listing the vertices and the faces, then create a mesh representing the shape as a set of vectors. The last step is to save the data as an STL file. The code is as follows:

In [ ]:
# Define the 8 vertices of the cube as point [ ±1, ±1, ±1 ]
vertices = np.array([\
    [-1, -1, -1], #0, first of four on the bottom face
    [+1, -1, -1], #1
    [+1, +1, -1], #2
    [-1, +1, -1], #3
    [-1, -1, +1], #4, first of four on the top face
    [+1, -1, +1], #5
    [+1, +1, +1], #6
    [-1, +1, +1]  #7
    ])
# Take the six faes of the cube and express as two triangles each
faces = np.array([\
    [0,3,1], [1,3,2], # bottom face
    [4,5,6], [4,6,7], # top face
    [0,4,7], [0,7,3], # left front face
    [0,1,5], [0,5,4], # right front face
    [2,3,6], [3,7,6], # left back face
    [5,1,2], [5,2,6]  # right back face
    ])

# Create the mesh
cube = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
for i, f in enumerate(faces):
    for j in range(3):
        cube.vectors[i][j] = vertices[f[j],:]

# Write the mesh to file "cube.stl"
cube.save('cube.stl')

## Third example. A mathematically defined surface (hyperboloid)

Here we demonstrate how to print a surface given from a mathematical function.

Let's start with a visual plot of a surface, say a hyperboloid given by the equation $$ z = x^2 - y^2. $$

We start with an rectangular grid in the x-y plane, create a meshgrid, define the z-values from the function, and then plot on the screen. 

Here is the code:

In [ ]:
# Code for a 3D surface plot

# set the number of points in x and y directions
Nx = 10
Ny = 10

x = np.linspace(-1,1,Nx)
y = np.linspace(-1,1,Ny)

# create a mesh grid in x and y, then define z values.
X, Y = np.meshgrid(x, y)
Z = X*X-Y*Y

# now we plot
fig = plt.figure(figsize = (10,10))
ax = plt.axes(projection='3d')
psurf = ax.plot_surface(X, Y, Z)
ax.set_xlabel('x', labelpad=20)
ax.set_ylabel('y', labelpad=20)
ax.set_zlabel('z', labelpad=20)

plt.show()

## Triangulating the surface.

To triangulate the surface, we break up each cell in the grid as a union of two triangles, oriented counterclockwise. 

<div align="center">
<img src="images/Cell_triangles.png" alt="A grid cell with two triangles" width="400"/><br>
A grid cell with two oriented triangles.
</div>

The coordinates of the three vertices in each triangle are fed into the **vectors** dataset of the 3D print mesh. Noting the coordinates for x,y in the diagram above, the three vertices of the red triangle are given as

```
bottom left: [ X[i,j],   Y[i,j],     Z[i,j] ]
  top right: [ X[i+1,j+1],Y[i+1,j+1],Z[i+1,j+1] ]
   top left: [ X[i+1,j],  Y[i+1,j],  Z[i+1,j] ],
```
while the coordinates for the blue triangle are given as
```
 bottom left: [ X[i,j],    Y[i,j],    Z[i,j] ]
bottom right: [ X[i,j+1],  Y[i,j+1],  Z[i,j+1] ]
   top right: [ X[i+1,j+1],Y[i+1,j+1],Z[i+1,j+1] ].
```

The 3D printer expects to see a bottom surface as well, so we get set $Z0 = Z - h $ to get a second surface a distance $h$ below the first. Here is how the two surfaces look together, with $h$ set to a large value for clarity. 
<div align="center">
<img src="images/Two_surfaces.png" alt="Two surfaces, top and bottom" width="400"/><br>
Two surfaces, top and bottom.
</div>
This second surface also must be triangulated and vectors added to the mesh. Since it is on the bottom, the oriented triangles go in the opposite directions 

Finally, we need to connect the top and bottom surfaces along their edges in order to give a solid shape with vertical walls along the edges. This figure shows two of the edge walls built; there are four in all. 
<div align="center">
<img src="images/Two_surfaces_wall.png" alt="Two surfaces, with walls at the edge" width="400"/><br>
Two surfaces, with walls at the edge.
</div>

Finally, we write the code to create the surface mesh that is used to create the STL file for printing. Note that the arrays $X,Y,Z$ define points on our vertices and we don't need to explicitly label the faces. This makes the code easier to understand, as we just loop over grid points to create the final data structure. 

The two surfaces have $Nx*Ny$ points each, giving $(Nx-1)*(Ny-1)$ rectangles each, and so we have $4(Nx-1)*(Ny-1)$ triangles for their triangulation. The walls give another $4(Nx-1) + 4(Ny-1)$ triangles, so the total number of triangles (faces) is
$$\mbox{Nfaces } = 4(Nx-1)(Ny-1) + 4(Nx-1) + 4(Ny-1).$$

Here is the code:

In [ ]:
h = 0.1  # thickness for the solid surface
Z0 = Z - h # The bottom surface. Could also use Z0 = np.min(Z) for a flat base. 

Nfaces = 4*(Nx-1)*(Ny-1) + 4*(Nx-1) + 4*(Ny-1)
surf = mesh.Mesh(np.zeros(Nfaces, dtype=mesh.Mesh.dtype))
k = 0

# top surface grid, two triangles per rectangular cell
for i in range(Ny-1):
    for j in range(Nx-1):
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z[i,j] ]
        surf.vectors[k][1] = [ X[i+1,j+1],Y[i+1,j+1],Z[i+1,j+1] ]
        surf.vectors[k][2] = [ X[i+1,j],Y[i+1,j],Z[i+1,j] ]
        k +=1
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z[i,j] ]
        surf.vectors[k][1] = [ X[i,j+1],Y[i,j+1],Z[i,j+1] ]
        surf.vectors[k][2] = [ X[i+1,j+1],Y[i+1,j+1],Z[i+1,j+1] ]
        k += 1
# bottom surface grid, two triangles per rectangular cell. Note clockwise order!
h = 0.1
for i in range(Ny-1):
    for j in range(Nx-1):
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z0[i,j] ]
        surf.vectors[k][2] = [ X[i+1,j+1],Y[i+1,j+1],Z0[i+1,j+1] ]
        surf.vectors[k][1] = [ X[i+1,j],Y[i+1,j],Z0[i+1,j] ]
        k +=1
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z0[i,j] ]
        surf.vectors[k][2] = [ X[i,j+1],Y[i,j+1],Z0[i,j+1] ]
        surf.vectors[k][1] = [ X[i+1,j+1],Y[i+1,j+1],Z0[i+1,j+1] ]
        k += 1

# now we create the side walls. Each wall connects the bottom surface Z0 to the top Z
# each wall has a line of rectangular cells which become two triangles each

# lower x side wall, two triangles for each of Nx-1 cells on edge
for i in [0]:
    for j in range(Nx-1):
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z0[i,j] ]
        surf.vectors[k][1] = [ X[i,j+1],Y[i,j+1],Z[i,j+1] ]
        surf.vectors[k][2] = [ X[i,j],Y[i,j],Z[i,j] ]
        k +=1
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z0[i,j] ]
        surf.vectors[k][1] = [ X[i,j+1],Y[i,j+1],Z0[i,j+1] ]
        surf.vectors[k][2] = [ X[i,j+1],Y[i,j+1],Z[i,j+1] ]
        k += 1
# upper x side wall, note the clockwise order!
for i in [Ny-1]:
    for j in range(Nx-1):
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z0[i,j] ]
        surf.vectors[k][2] = [ X[i,j+1],Y[i,j+1],Z[i,j+1] ]
        surf.vectors[k][1] = [ X[i,j],Y[i,j],Z[i,j] ]
        k +=1
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z0[i,j] ]
        surf.vectors[k][2] = [ X[i,j+1],Y[i,j+1],Z0[i,j+1] ]
        surf.vectors[k][1] = [ X[i,j+1],Y[i,j+1],Z[i,j+1] ]
        k += 1
# left y side wall, two triangles for each of Ny-1 cells on edge
for i in range(Ny-1):
    for j in [0]:
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z0[i,j] ]
        surf.vectors[k][1] = [ X[i+1,j],Y[i+1,j],Z[i+1,j] ]
        surf.vectors[k][2] = [ X[i,j],Y[i,j],Z[i,j] ]
        k +=1
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z0[i,j] ]
        surf.vectors[k][1] = [ X[i+1,j],Y[i+1,j],Z0[i+1,j] ]
        surf.vectors[k][2] = [ X[i+1,j],Y[i+1,j],Z[i+1,j] ]
        k += 1
# right y side wall, note the clockwise order!
for i in range(Ny-1):
    for j in [Nx-1]:
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z0[i,j] ]
        surf.vectors[k][2] = [ X[i+1,j],Y[i+1,j],Z[i+1,j] ]
        surf.vectors[k][1] = [ X[i,j],Y[i,j],Z[i,j] ]
        k +=1
        surf.vectors[k][0] = [ X[i,j],Y[i,j],Z0[i,j] ]
        surf.vectors[k][2] = [ X[i+1,j],Y[i+1,j],Z0[i+1,j] ]
        surf.vectors[k][1] = [ X[i+1,j],Y[i+1,j],Z[i+1,j] ]
        k += 1


# Write the mesh to file "surf.stl"
surf.save('surf.stl')

## Viewing your results

You now have a file in your Jupyter hub called surf.stl that you can view and print. Download this file to your local computer, then drag and drop into an online STL viewer such as https://www.viewstl.com/

You can also view the Pyramid and Cube files created earlier by dragging them to the same site. 


## 3D Printing the file - final steps

You now have a file in your Jupyter hub called surf.stl that you can print. Download this file to your local computer, then open it with your favourite 3D printing software. For instance, I am using PrusaSlicer, with is based on Slic3r. Use the Slicer software to adjust the size, add supports if necessary and adjust any other settings for the filament type and quality of print you require. Then create a g-code file and transfer this to your 3D printer. 

And print!  

Here is a sample result, using PLA filament in draft mode on a Prusa printer.

<div align="center">
<img src="images/Printed_surface.jpg" alt="The final printed surface" width="400"/><br>
The final printed surface.
</div>

## Going further

We can try several ideas to explore this 3D printing:

- try printing the surface rotated on its side, so fewer supports are needed to print
- increase the number of grid points to obtain a smoother printed surface
- use a different function to get a different surface
  - eg, a paraboloid is given by the equation $z = x^2 + y^2$
  - an elliptical parboloid is given by $z = x^2 + 4y^2$
  - a wavey function is given by $z = \sin(10*(x^2 + y^2))$
- modify the code to use a circular grid instead of a rectangular grid, using polar coordinates

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)